</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Этапы выполнения проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.



**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Подготовка

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re 
from nltk.corpus import stopwords 
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

import time

from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from scipy.stats import uniform


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
path_1 = '/content/12_toxic_comments.csv'
path_2 = '/datasets/toxic_comments.csv'
try:
    df = pd.read_csv(path_1)
except:
    df = pd.read_csv(path_2)

In [ ]:
SEED = 12345

In [ ]:
df.head(20) #посмотрим на общий вид данных в датасете

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


В датасете мы видим колонку с комментариями и колонку с классификацией комментария по признаку его тксичности ( 1 - комментарий токсичный, 0 - комментарий нейтральный).

In [ ]:
df.info() #изучим общую информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Данные содержат около 160 К строк, в колонке с текстом тип данных `object`, в колонке c классификатором токсичности комментария данные целочисленные.

In [ ]:
df['toxic'].value_counts() # посмотрим на распределение значений целевого признака

0    143106
1     16186
Name: toxic, dtype: int64

In [ ]:
print (f"Доля токсичных комментариев: {round(len(df[df['toxic']==1])/len(df)*100,1)} %")

Доля токсичных комментариев: 10.2 %


Доля токсичных комментариев составляет чуть больше 10%.

На следующем этапе перейдем к подготовке текста для последующего обучения моделей.

In [ ]:
def clear_text(text): # создадим функцию для очистки текстовых данных от всех символов, кроме латинских букв и пробелов 
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [ ]:
corpus = df['text'] #создадим функцию по лемматизации данных
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    return " ".join([lemmatizer.lemmatize(w) for w in word_list])

corpus_lemm = [lemmatize(clear_text(corpus[i])) for i in range(len(corpus))] #проведем лемматизацию текста
corpus_lemm # выведем полученный лемматизированный текст на экран

['Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now',
 'D aww He match this background colour I m seemingly stuck with Thanks talk January UTC',
 'Hey man I m really not trying to edit war It s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info',
 'More I can t make any real suggestion on improvement I wondered if the section statistic should be later on or a subsection of type of accident I think the reference may need tidying so that they are all in the exact same format ie date format etc I can do that later on if no one else doe first if you have any preference for formatting style on reference or want to do it yourself please let me know There appears to be

In [ ]:
stop_words = set(stopwords.words('english')) #обозначим множество наиболее часто встречающихся слов, не влияющих на тональность текста

In [ ]:
y = df['toxic'] #выделим данные по целевому признаку в датасете

In [ ]:
X_train_, X_test_, y_train, y_test = train_test_split(corpus_lemm, y, test_size =.2, stratify = y) # разделим выборку на тренировочную и тестовую из соотношения 80:20
#print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words) #проведем векторизацию данных, для этого создадим счетчик, указав в нем стоп-слова

In [ ]:
X_train = count_tf_idf.fit_transform(X_train_) #из полученных векторных данных выделим тренировочные и тестовые признаки
X_test = count_tf_idf.transform(X_test_)

## Обучение

На данном этапе подберем наилучшие гиперпараметры для моделей, а затем обучим каждую из них.
Для сравнения возьмем следующие модели:
* LogisticRegression;
* RandomForestClassifier;
* LinearSVC;
* CatBoostClassifier.

In [ ]:
def choose_model(model): # во избежание дублирования кода, напишем функцию расчета тренировочного времени и метрики F1 для заданной модели
    start = time.time()
    model.fit(X_train_,y_train)
    train_time = time.time()-start
    predict_test = model.predict(X_test_)
    return f1_score(predict_test, y_test), train_time

Первой рассмотрим логистическую регрессию и подберем для нее наилучшие гиперпараметры.

In [ ]:
pipe_lr = Pipeline([('tfidfvect', TfidfVectorizer(stop_words=stop_words)),
                   ('clf', LogisticRegression(random_state=SEED))])
distributions ={'clf__C':np.linspace(10,20,10, endpoint = True)} #dict(C=uniform(loc=0, scale=4))#
grid_lr = RandomizedSearchCV(pipe_lr,
                               distributions,
                               cv=5,
                               n_jobs=-1,
                               scoring = 'f1')#LogisticRegression(max_iter = 1000, random_state=SEED)
grid_lr.fit(X_train_,y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidfvect',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                 

In [ ]:
print (f'Лучшие гиперпараметры для логистической регрессии: {grid_lr.best_params_}')

Лучшие гиперпараметры для логистической регрессии: {'clf__C': 18.88888888888889}


In [ ]:
res = [] 
res.append(choose_model(grid_lr.best_estimator_)) #время тренировки модели и метрику F1 добавим в общий список


Подберем параметры для случайного леса.

In [ ]:
pipe_rfc = Pipeline([('tfidfvect', TfidfVectorizer(stop_words=stop_words)),
                   ('rfc', RandomForestClassifier(random_state=SEED))])

parametrs = { 'rfc__n_estimators': range (10, 71, 5),
              'rfc__max_depth': range (1,50, 2),
             }
grid_rfс = RandomizedSearchCV(pipe_rfc, 
                              parametrs, 
                              cv=5, 
                              n_jobs=-1,
                              scoring='f1', 
                              verbose = False)
grid_rfс.fit(X_train_,y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidfvect',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                 

In [ ]:
print (f'Лучшие гиперпараметры для RandomForestClassifier:{grid_rfс.best_params_}')

Лучшие гиперпараметры для RandomForestClassifier:{'rfc__n_estimators': 40, 'rfc__max_depth': 49}


In [ ]:
res.append(choose_model(grid_rfс.best_estimator_)) #время тренировки модели и метрику F1 добавим в общий список

Следующей рассмотрим модель LinearSVC.

In [ ]:
pipe_lsvc = Pipeline([('tfidfvect', TfidfVectorizer(stop_words=stop_words)),
                   ('lsvc', LinearSVC(max_iter = 1000, random_state=SEED))])
parametrs = {'lsvc__C':np.linspace(1,30,7,endpoint=True)}
grid_lsvc = RandomizedSearchCV(pipe_lsvc,
                               parametrs,
                               cv=5, 
                               n_jobs=-1,
                               scoring = 'f1')
grid_lsvc.fit(X_train_,y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidfvect',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                 

In [ ]:
print (f'Лучшие гиперпараметры для LinearSVC:{grid_lsvc.best_params_}')

Лучшие гиперпараметры для LinearSVC:{'lsvc__C': 1.0}


In [ ]:
res.append(choose_model(grid_lsvc.best_estimator_)) #время тренировки модели и метрику F1 добавим в общий список

Последней рассмотрим модель CatBoostClassifier.

In [ ]:
pipe_cbc = Pipeline([('tfidfvect', TfidfVectorizer(stop_words=stop_words)),
                   ('cbc', CatBoostClassifier(random_state=SEED))])
parametrs = {'cbc__iterations': range(40,80,10)}
grid_cbc = RandomizedSearchCV(pipe_cbc,
                               parametrs,
                               cv=5, 
                               n_jobs=-1,
                               scoring = 'f1')
grid_cbc.fit(X_train_,y_train)

Learning rate set to 0.5
0:	learn: 0.3467309	total: 1.78s	remaining: 2m 2s
1:	learn: 0.2616076	total: 3.32s	remaining: 1m 53s
2:	learn: 0.2360668	total: 4.85s	remaining: 1m 48s
3:	learn: 0.2212403	total: 6.46s	remaining: 1m 46s
4:	learn: 0.2126975	total: 8.8s	remaining: 1m 54s
5:	learn: 0.2073966	total: 10.3s	remaining: 1m 50s
6:	learn: 0.2022282	total: 11.8s	remaining: 1m 46s
7:	learn: 0.1959705	total: 13.4s	remaining: 1m 44s
8:	learn: 0.1919292	total: 14.9s	remaining: 1m 41s
9:	learn: 0.1889815	total: 16.4s	remaining: 1m 38s
10:	learn: 0.1839642	total: 18s	remaining: 1m 36s
11:	learn: 0.1813897	total: 19.4s	remaining: 1m 34s
12:	learn: 0.1790568	total: 20.9s	remaining: 1m 31s
13:	learn: 0.1766655	total: 22.9s	remaining: 1m 31s
14:	learn: 0.1747223	total: 24.8s	remaining: 1m 30s
15:	learn: 0.1714550	total: 26.3s	remaining: 1m 28s
16:	learn: 0.1697849	total: 27.8s	remaining: 1m 26s
17:	learn: 0.1683882	total: 29.2s	remaining: 1m 24s
18:	learn: 0.1669389	total: 30.7s	remaining: 1m 22s
1

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidfvect',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                 

In [ ]:
print (f'Лучшие гиперпараметры для CatBoostClassifier:{grid_cbc.best_params_}')

Лучшие гиперпараметры для CatBoostClassifier:{'cbc__iterations': 70}


In [ ]:
res.append(choose_model(grid_cbc.best_estimator_)) #время тренировки модели и метрику F1 добавим в общий список

Learning rate set to 0.5
0:	learn: 0.3467309	total: 1.73s	remaining: 1m 59s
1:	learn: 0.2616076	total: 3.27s	remaining: 1m 51s
2:	learn: 0.2360668	total: 4.8s	remaining: 1m 47s
3:	learn: 0.2212403	total: 6.39s	remaining: 1m 45s
4:	learn: 0.2126975	total: 7.88s	remaining: 1m 42s
5:	learn: 0.2073966	total: 9.41s	remaining: 1m 40s
6:	learn: 0.2022282	total: 10.9s	remaining: 1m 38s
7:	learn: 0.1959705	total: 12.5s	remaining: 1m 37s
8:	learn: 0.1919292	total: 14.1s	remaining: 1m 35s
9:	learn: 0.1889815	total: 15.6s	remaining: 1m 33s
10:	learn: 0.1839642	total: 17.1s	remaining: 1m 31s
11:	learn: 0.1813897	total: 18.6s	remaining: 1m 29s
12:	learn: 0.1790568	total: 20.1s	remaining: 1m 28s
13:	learn: 0.1766655	total: 21.6s	remaining: 1m 26s
14:	learn: 0.1747223	total: 23.1s	remaining: 1m 24s
15:	learn: 0.1714550	total: 24.7s	remaining: 1m 23s
16:	learn: 0.1697849	total: 26.2s	remaining: 1m 21s
17:	learn: 0.1683882	total: 27.6s	remaining: 1m 19s
18:	learn: 0.1669389	total: 29.1s	remaining: 1m 18

## Выводы

Для наглядного сравнения, данные по времени тренировки и метрике F1 для каждой модели сведем в одну таблицу.

In [ ]:
pd.DataFrame(res,index =['LR','RFC','LinearSVC','CatBooster'], columns=['F1','train_time'])

,F1,train_time
LR,0.770388,9.513475
RFC,0.070322,17.149848
LinearSVC,0.774116,5.877884
CatBooster,0.729282,135.884296


На основе анализа приведенной таблицы, можем сделать вывод, что самой точной является модель LinearSVC, при этом данная модель также быстрее всего обучается.

В ходе проведенной работы мы изучили данные с комментариями пользователей интернет-магазина "Викишоп", при этом было определено, что доля негативных комментариев составляет около 10% от общего числа отзывов.

Для обучения моделей мы лемматизировали данные, избавили их от лишних символов, а также от слов не несущих смысловой нагрузки ( артиклей, союзов, предлогов и т.п.). Очищенный и лемматизированный текст мы привели к векторному виду при помощи TfidfVectorizer, полученные признаки использовали для подбора наилучшей модели по метрике качества `F1`. 

Было обучено 4 модели:
* LogisticRegression;
* RandomForestClassifier;
* LinearSVC;
* CatBoostClassifier.

Наилучшая метрика качества принадлежит LinearSVC.
Будем рекомендовать данную модель использовать для классификации комментариев на негативные и позитивные.